In [23]:
#@title Connecting to google drive 🥇
from google.colab import drive
drive.mount('/content/drive/')


from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


# importing libraries
import pandas as pd
import shutil
import os
import datetime 
import numpy as np
#
import re

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
#@title Main Code


file_name = 'Orders_Summary_2023_04_29_15_23_21'   # paste file name here




# ******************************************************************************* MAIN CODE **************************************************************************************

file_path = f'/content/drive/MyDrive/MFC1089_Ehthesham_Working_Folder/Order wise sales/{file_name}.xlsx'


df=pd.read_excel(f'{file_path}',skiprows=range(9),names=["Invoice No.",
  "Date","Biller","KOT No.","Payment Type","Order Type","Status","Area","Sub Order Type","Group Name","Assign To","Phone","Name","Address","Persons",
  "Order Cancel Reason","My Amount (₹)","Discount (₹)","Net Sales (₹)(M.A - D)","Delivery Charge","Container Charge","Service Charge","Total Tax (₹)","Round Off",
  "Waived off","Total (₹)","Online Tax Calculated","GST Paid by Merchant","GST Paid by Ecommerce","Tip (₹)","Non Taxable","Amount (SGST)","SGST","Amount (CGST)","CGST"])


new_df=pd.DataFrame()


new_df['Invoice Date']=df['Date']
new_df['Invoice Status']='Overdue'



new_df["GST Treatment"]  ='Consumer'

new_df["Place of Supply"]  ='KA'

new_df["Due Date"]  =df['Date']


new_df["Currency Code"]  ='INR'
new_df["Exchange Rate"]  =1
new_df["Account"]  ='Sales'
new_df["Item Name"]  =df['Payment Type']

new_df["Item Type"]  ='Goods'

new_df["Quantity"]  =1
new_df["Usage unit"]  ='Count'
new_df["Item Price"]  = df['My Amount (₹)']

new_df["Is Inclusive Tax"]  = 'Tax'
new_df["Item Tax"]  = 'GST5'
new_df["Item Tax Type"]  = 'Tax Group'
new_df["Item Tax %"]  = '5%'



new_df["Supply Type"]  = 'Taxable'
new_df["Discount Type"]  = 'entity_level'


new_df["Discount Amount"]  = df['Discount (₹)']

new_df["Shipping Charge"]  = df['Container Charge']

new_df['HSN/SAC']='996331'

new_df["Adjustment"]  = 'Adjustment'
new_df["Total Amount"]  = df['Total (₹)']


#@title map
p={"HHH" :	"Biryani HUB"	,
"BBB" :	"Biryani Box"	,
"DDD" :	"Dilli Darbar"	,
"BBB" :	"Biryani Box"	,
"AAA" :	"Alpha Biryani"	,
"RRR" :	"Roj Ka Khana"	,
"CCC" :	"Chinese Chaska"	,
"VVV" :	"Valluvan Cafe"	,
"MMM" :	"Mann Food Court (NEW)"	,
"UUU" :	"URUVALU BIRYANI"	,
"JJJ" :	"MFC Juice & Cafe"	,
"KKK" :	"Koolerz"	,
"GGG" :	"Grill Factory"	,
"TTT" :	"Dilli Chaat"	}

new_df['Brand']=df['Area'].str.split('_').str[1]
mask = new_df['Brand'].apply(lambda x : len(x)> 3 if isinstance(x, str) else False)

# Use the mask to assign NaN to cells that meet the condition
new_df.loc[mask, 'Brand'] = np.nan
new_df['Brand'] = new_df['Brand'].map(lambda x: p[x] if x in p else x)


#@title extra
new_df['Platform']=df['Area'].str.split("_",expand=True)[0]
new_df['payment_type']= new_df['Brand'].fillna(df['Payment Type'])

new_df['Platform'] = new_df['Platform'].where(new_df['Platform'].isin(['Zomato', 'Swiggy', 'Hashtag']))
new_df['Customer Name']=new_df['Platform']+" "+new_df['payment_type']
new_df['Customer Name']=new_df['Customer Name'].fillna(df['Payment Type'])

# if item Nmae = 'Online' then Item Tax % = 0% and supply type = 'Exempted' else 5% and 'Taxable' 
new_df['Item Price']=new_df['Total Amount']
mask = new_df['Item Name'].apply(lambda x : True if x=='Online' else False)
new_df.loc[mask, 'Item Tax %'] = '0%'
new_df.loc[mask, 'Supply Type'] = 'Exempted'

#convert to date
new_df['Invoice Date']=new_df['Invoice Date'].str.split(" ",expand=True)[0]
new_df['Invoice Date']=pd.to_datetime(new_df['Invoice Date'])
new_df['Day']=new_df['Invoice Date'].dt.day



# new_df.groupby(['Day','Customer Name','Invoice Status', 'GST Treatment', 'Place of Supply',
#         'Currency Code', 'Exchange Rate', 'Account', 'Item Name',
#        'Item Type','Quantity','Usage unit','Is Inclusive Tax',
#        'Item Tax','Item Tax Type','Item Tax %','Supply Type',
#        'Discount Type','Adjustment'])[['Quantity','Item Price','Discount Amount','Shipping Charge','Total Amount']].sum().to_excel('Zoho Sales Export Mar-2023.xlsx')


#new_df['Customer Name'].unique() #

new_df_only_swig_zom_thrive=new_df[~new_df['Customer Name'].isin(['Other [HDFC BharatQR](NEW GUEST REQUESTED 3 ,)', 'Due Payment',
       'Other [HDFC BharatQR](40)','Other [HDFC BharatQR](aiepi@0367)','Other [Approved By MD]','Other [HDFC UPI]','Other [Magic Pay]','Online','Cash','Part Payment', 'Other [PhonePe]',
       'CARD', 'Other [HDFC BharatQR]'])]


swig_zom_thrive_sales_grouped=new_df_only_swig_zom_thrive.groupby(['Day','Invoice Date','Customer Name','Invoice Status', 'GST Treatment', 'Place of Supply',
        'Currency Code', 'Exchange Rate', 'Account', 'Item Name',
       'Item Type','Quantity','Usage unit','Is Inclusive Tax',
       'Item Tax','Item Tax Type','Item Tax %','Supply Type',
       'Discount Type','Adjustment'])[['Quantity','Item Price','Discount Amount','Shipping Charge','Total Amount']].sum()


swig_zom_thrive_sales_grouped_db=pd.DataFrame(swig_zom_thrive_sales_grouped)
swig_zom_thrive_sales_grouped_db = swig_zom_thrive_sales_grouped_db.rename(columns={"Quantity": "NewName"})

swig_zom_thrive_sales_grouped_db=swig_zom_thrive_sales_grouped_db.reset_index()
swig_zom_thrive_sales_grouped_db = swig_zom_thrive_sales_grouped_db.rename(columns={"NewName": "Quantity"})


swig_zom_thrive_sales_grouped_db.head(2)

,Day,Invoice Date,Customer Name,Invoice Status,GST Treatment,Place of Supply,Currency Code,Exchange Rate,Account,Item Name,...,Item Tax Type,Item Tax %,Supply Type,Discount Type,Adjustment,Quantity,Item Price,Discount Amount,Shipping Charge,Total Amount
0,1,2023-04-01,Hashtag Grill Factory,Overdue,Consumer,KA,INR,1,Sales,Online,...,Tax Group,0%,Exempted,entity_level,Adjustment,1,491,448.0,20.0,491
1,1,2023-04-01,Hashtag Mann Food Court (NEW),Overdue,Consumer,KA,INR,1,Sales,Online,...,Tax Group,0%,Exempted,entity_level,Adjustment,4,2163,513.4,110.0,2163


In [25]:
#@title Save file to folder

Give_name_for_file='Zoho sales Apr01-Apr28_2023' # type ur file name here


# ************************* --------------------------------------------
swig_zom_thrive_sales_grouped_db.to_excel(f'/content/drive/MyDrive/MFC1130_Jamshed_Working_Folder/Zoho Sales Export /{Give_name_for_file}.xlsx',index=False)